# Programación orientada a objetos

Ya hemos discutido sobre los tipos de colecciones en Python, funcionan mas o menos así:
1. El tipo puede ser llamado como función para crear nuevas **instancias**.
2. La llamada al tipo puede incluir argumentos usados para inicializar la nueva instancia.
3. Podemos invocar **métodos** sobre instancias de los tipos en lugar de inspeccionar y manipular por su representación interna.
4. Métodos para distintos tipos pueden tener el mismo nombre.
5. La forma en que se imprime una instancia se basa en su tipo.

Nada de lo anterior es especial para los tipos de colecciones, en realidad, todas estas características son parte de un mecanismo incorporado a Python para organizar ideas y modelar problemas.

Podemos crear un nuevo tipo en Python con una definición de **clase**, similar a como podemos definir una nueva función.

Veamos un ejemplo y luego discutamos algunas propiedades de estas técnicas de programación.

# La tarea

1. Modela este problema utilizando técnicas de orientado a objetos.

In [ ]:
import random
import time
import math

In [316]:
class Itinerario:
    def __init__(self, state, domain):
        self.state = state
        self.domain = domain

    def neighbors(self):
        ''' Regresa una lista de estados en forma de solución
        que difieren en un vuelo con respecto al estado inicial
        '''
        neighbors = []
        s = self.solucion
        for i in range(len(self.domain)):
            
            if s[i] > self.domain[i][0]:
                neighbors.append(Solucion(s[0:i] + [s[i] - 1] + s[i+1:]))
            if s[i] < self.domain[i][1]:
                neighbors.append(Solucion(s[0:i] + [s[i] + 1] + s[i+1:]))
        return neighbors

    def __str__(self):
        return str(self.state)

    def __repr__(self):
        return self.__str__()

    def __iter__(self):
        return self.state

    def __len__(self):
        return len(self.state)
        
    def __getitem__(self, indice):
        return self.state[indice]

    def mutate(self):
        '''Regresa un vecino'''
        return random.choice(self.neighbors()) 
        
    def crossover(self, s2):
        '''Regreso una mezcla de dos estados'''
        i = random.randint(1, len(self)-2)
        return Itinerario(self[:i] + s2[i:])

In [ ]:
class Flights:
    def __init__(self, path_flights, people, destination):
        self.destination = destination
        self.people = people
        self.flights = {}
        
        with open(path) as f:
            for line in f:
                origin, dest, t_depart, t_arrive, price = line.strip().split(',')
                self.flights.setdefault((origin, dest), [])
                self.flights[(origin, dest)].append((
                    self.get_minutes(t_depart), 
                    self.get_minutes(t_arrive), 
                    int(price)
                ))
    def show_state(self, s):
        '''Representacion legible de un estado'''
        assert isinstance(s, Itinerario), 's debe ser un objeto tipo Itinerario'
        for i in range(len(s) // 2):
        name = people[i][0]
        origin = people[i][1]
        out = flights[(origin,destination)][s[2*i]]
        ret = flights[(destination,origin)][s[2*i+1]]
        print('%10s%10s %5s-%5s $%3s %5s-%5s $%3s' % (
            name,
            airports[people[i][1]][2],
            out[0],out[1],out[2],
            ret[0],ret[1],ret[2],
        ))
            
    
    def get_dep(self, nombre):
        if isinstance(nombre, str):
            for tupla in self.people:
                if tupla[0] == nombre:
                    return tupla[1]
        if isinstance(nombre, int):
            return self.people[nombre][1]
        

    def get_flights_dep(self, nombre):
        dep = self.get_dep(nombre)
        
        return self.flights[(dep,self.destination)]
            
    def get_flights_ret(self, nombre):
        dep = self.get_dep(nombre)
        return self.flights[(self.destination, dep)]
            
    def random_itinerio(self):
        '''Regresa un itinerario aleatorio'''
        itinerio = []
        for tupla in self.people:
            nombre = tupla[0]
            itinerio.append(
                random.choice(range(len(self.get_flights_dep(nombre))))
            )
            itinerio.append(
                random.choice(range(len(self.get_flights_ret(nombre))))
            )
        return Itinerario(itinerio)

    def get_minutes(self, t):
        x = time.strptime(t, '%H:%M')
        h = x.tm_hour
        m = x.tm_min
        return 60 * h + m
    
    def get_hours(self, t):
        h = t // 60
        m = t - h * 60 
        h = str(h).rjust(2, '0')       
        m = str(m).rjust(2, '0')
        return h + ':' + m
        
        
    def schedule_cost(self, solucion):
        # contamos el precio total de cada vuelo (ida y regreso)
        total_price = 0
        s = solucion
        # nos interesa conocer el tiempo de llegada a NY mas tarde
        # y el tiempo de salida de NY mas temprano.
        latest_arrival = 0
        earliest_departure = 24 * 60
        
        for i in range(len(s) // 2):
            origin = self.people[i][1]
            out_flight = self.flights[(origin, self.destination)][s[2*i]]
            ret_flight = self.flights[(self.destination, origin)][s[2*i+1]]
            
            total_price += out_flight[2] # vuelo de ida
            total_price += ret_flight[2] # vuelo de regreso
            
            # tiempo de llegada máximo
            # tiempo de salida mínimo
            if latest_arrival < out_flight[1]:
                latest_arrival = out_flight[1]
            if earliest_departure > ret_flight[0]:
                earliest_departure = ret_flight[0]
        
        # contamos el tiempo de espera de cada persona
        total_wait = 0
        
        for i in range(len(s) // 2):
            origin = self.people[i][1]
            out_flight = self.flights[(origin, self.destination)][s[2*i]]
            ret_flight = self.flights[(self.destination, origin)][s[2*i+1]]
            
            # todos esperan al último familiar en llegar
            total_wait += latest_arrival - out_flight[1]
            
            # todos llegan al aeropuerto al mismo tiempo y esperan su vuelo
            total_wait += ret_flight[0] - earliest_departure
            
            # si el último en llegar a NY llega después del primero en
            # irse de NY se paga un día más de la renta del carro.
            # el costo de la renta por un día es independiente de la
            # solución.
            if latest_arrival > earliest_departure:
                total_price += 50
        
        # El costo total es el precio total de los vuelos y el tiempo de
        # espera total de las personas.
        # Buscamos soluciones con un bajo costo.
        return total_price + total_wait

In [ ]:
class Algorithms:
    def __init__(self, flights, fun_cost = None):
        self.flights = flights
        
        if not fun_cost:
            self.fun_cost = self.flights.schedule_cost
        else:
            self.fun_cost = fun_cost 
        
        
    def solve_randomly(self, repeats = 1000):
        '''Devuelve el estado con coste más bajo encontrado
        aleatoriamente'''
        best_cost = float('inf')
        best_sol = None
        
        for _ in range(repeats):
            s = self.flights.random_state()
            c = self.fun_cost(s)
            if c < best_cost:
                best_cost = c
                best_sol = s
        
        return best_sol

    def solve_hillclimbing(self, repeats = 1000, init_state = None):
        '''Devuelve el estado con coste más bajo encontrado con hill climbing'''
        if  init_state is None:
            s = self.flights.random_state()
        else:
            s = init_state
        contador = 0
        while (True and contador <= 1000):
            contador = contador + 1
            neighbors = s.neighbors()
            cost = self.fun_cost(s)
            best_neighbor = min(neighbors, key=self.fun_cost)
            neighbor_cost = self.fun_cost(best_neighbor)
            
            if cost <= neighbor_cost:
                return s
            
            s = best_neighbor
    def solve_annealing(self, Ti=10000.0, Tf=0.1, alpha=0.95, init_state = None):
        '''Devuelve el estado con coste más bajo encontrado con annealing'''
        cost_of = self.fun_cost
        if  init_state is None:
            itinerario = self.flights.random_state()
        else:
            itinerario = init_state
        cost = cost_of(itinerario)
        T = Ti
        while T > Tf:
            neighbor = random.choice(itinerario.neighbors())
            neighbor_cost = cost_of(neighbor)
            diff = cost - neighbor_cost
            if diff > 0 or random.random() < math.exp(diff / T):
                itinerario = neighbor
                cost = neighbor_cost
            T = alpha*T
        
        return itinerario

    def solve_evolving(self, pop_size=50, mut_prob=0.2, elite=0.2, epochs=100, init_state = None):
        '''Devuelve el estado con coste más bajo encontrado con evolving'''
        if  init_state is None:
            pop = [self.flights.random_state() for _ in range(pop_size)]
        else:
            pop = [self.flights.random_state() for _ in range(pop_size-1) ]
            pop.append(init_state)
        
        top_elite = int(elite * pop_size)
        
        for epoch in range(epochs):
            pop.sort(key=self.fun_cost)
            
            best = pop[:top_elite]
            while len(best) < pop_size:
                if random.random() < mut_prob:
                    best.append(best[random.randint(0, top_elite-1)].mutate())
                else:
                    best.append(best[random.randint(0, top_elite-1)].crossover(
                        best[random.randint(0, top_elite-1)])
                    )
            pop = best
        pop.sort(key=self.fun_cost)
        return pop[0]

2. Explora los parámetros y la función costo de este problema, intenta encontrar mejores valores para resolver el problema.

3. Elige un problema distinto, y modela las posibles soluciones del problema para utilizar los métodos discutidos en esta libreta: búsqueda de solución aleatoria, búsqueda de solución por descenso de colinas, búsqueda de solución por recocido simulado, búsqueda de solución por algoritmos genéticos.

In [317]:
import random
import math
import numpy as np
import matplotlib.pyplot as plt

In [318]:
import math
import random
import numpy as np

class SimplePendulum:
    def __init__(self, length, mass=1.0, gravity=9.8):
        self.length = length
        self.mass = mass
        self.gravity = gravity

        self.calculate_period()

    def calculate_period(self):
        self.period = 2 * math.pi * math.sqrt(self.length / self.gravity)

    def generate_random_trajectory(self, theta = 45, num_points = 100):
        trajectory = []
        length = self.length
        for _ in range(num_points):
            x = random.uniform(-length, length)
            y = random.uniform(-length, length)
            trajectory.append((x, y))

        trajectory[0] = (length*math.sin(theta*math.pi/180), length*math.cos(theta*math.pi/180))
        trajectory[-1] = (-length*math.sin(theta*math.pi/180), length*math.cos(theta*math.pi/180))
    
        return trajectory

    def generate_neighboring_trajectories(self, trajectory, factor = 0.1):
        num_neighbors = len(trajectory)
        neighbors = []
    
        for i in range(1, num_neighbors - 1):
            perturbed_trajectory = trajectory.copy()
            x, y = perturbed_trajectory[i]
            x, y =+ factor * random.uniform(-self.length, self.length), factor * random.uniform(-self.length, self.length)
            perturbed_trajectory[i] = (x, y)
            neighbors.append(perturbed_trajectory)
    
        return neighbors

    #Es otra manera de encontrar vecinos usando
    def generate_neighboring_trajectories_std(self, trajectory, num_neighbors=100):
        def calculate_standard_deviation(trajectory):
            points_x, points_y = zip(*trajectory)
            std_dev_x = np.std(points_x)
            std_dev_y = np.std(points_y)
            standard_deviation = (std_dev_x, std_dev_y)
            return standard_deviation
    
        standard_deviation = calculate_standard_deviation(trajectory)
        neighboring_trajectories = []
    
        for _ in range(num_neighbors):
            neighboring_trajectory = []
    
            for i, point in enumerate(trajectory):
                if i == 0 or i == len(trajectory) - 1:
                    continue
                
                displacement_x = np.random.normal(0, standard_deviation[0])
                displacement_y = np.random.normal(0, standard_deviation[1])
                
                neighbor_point = (point[0] + displacement_x, point[1] + displacement_y)
                neighboring_trajectory.append(neighbor_point)
                neighboring_trajectory[0] = (length*math.sin(theta*math.pi/180), length*math.cos(theta*math.pi/180))
                neighboring_trajectory[-1] = (-length*math.sin(theta*math.pi/180), length*math.cos(theta*math.pi/180))
    
            neighboring_trajectories.append(neighboring_trajectory)
    
        return neighboring_trajectories

    def calculate_action(self, trajectory):
        approximate_action = 0.0
        num_points = len(trajectory)
        time_interval = self.period / (2*num_points)

        for i in range(num_points - 1):
            x1, y1 = trajectory[i]
            x2, y2 = trajectory[i + 1]

            vx1 = (x2 - x1) / time_interval
            vy1 = (y2 - y1) / time_interval

            lagrangian = 0.5 * self.mass * (vx1**2 + vy1**2) - self.mass * self.gravity * y1
            approximate_action += lagrangian * time_interval

        return approximate_action

In [319]:
class Algorithms:
    def __init__(self, particula):
        self.particula = particula

    def best_random_trajectories(self, repeats = 1000):
        min_action = float('inf')
        best_trajectory = None
        
        for _ in range(repeats):
            traject = generate_random_trajectory(self)
            traject_action = particula.calculate_action(traject)
            if traject_action < min_action:
                min_action = traject_action
                best_trajectory = traject
                
        return traject

    def solve_hillclimbing(self):
        traject = generate_random_trajectory(particula, num_points = 1000)
        action = float('inf')
        
        while True:
            neighbors = generate_neighboring_trajectories(self, traject)
            action = particula.calculate_action(traject)
            best_neighbor = min(neighbors, key=particula.calculate_action)
            neighbor_action = particula.calculate_action(best_neighbor)
            
            if action < neighbor_action:
                return traject
            
            traject = best_neighbor

    def solve_annealing(self, Ti=10000.0, Tf=0.1, alpha=0.95):
        traject = self.generate_random_trajectory()
        action = self.calculate_action(traject)
        T = Ti
        while T > Tf:
            neighbor = random.choice(generate_neighboring_trajectories(traject))
            neighbor_action = self.calculate_action(neighbor)
            diff = cost - neighbor_action
            if diff > 0 or random.random() < math.exp(diff / T):
                traject = neighbor
                action = neighbor_action
            T = alpha*T
        
        return traject

    def solve_evolving(self, pop_size=50, mut_prob=0.2, elite=0.2, epochs=100):
        pop = [self.generate_random_trajectory() for _ in range(pop_size)]
        top_elite = int(elite * pop_size)
        
        for epoch in range(epochs):
            pop.sort(key=self.calculate_action)
            best = pop[:top_elite]
            while len(best) < pop_size:
                if random.random() < mut_prob:
                    best.append(mutate(
                        best[random.randint(0, top_elite-1)],
                        domain
                    ))
                else:
                    best.append(crossover(
                        best[random.randint(0, top_elite-1)],
                        best[random.randint(0, top_elite-1)],
                    ))
            pop = best
        pop.sort(key=cost_of)
        return pop[0]

In [320]:
pendulo = SimplePendulum(1)

4. Compara los resultados de las soluciones de los cuatro métodos anteriores.

5. ¿Es posible resolver el problema que planteas analizando todas las posibles soluciones? Justifica tu respuesta.

Sí, si consideramos que los estados estan descritos por una cantidad cantidad acotada de coordenadas que describen se consideran suficientes para describir de manera clara una trayectoria esta cantidad es estatica para una cantidad de coordenadas fija y una presición fija. Pero, el coste computacional de explorar todos los estados y encontrar aquel donde la acción sea minima parece ser muy grande como para considerarlo un
camino aceptable para encontrar una solución en tiempo razonable.